In [5]:
%load_ext autoreload
%autoreload 2

import os 
import numpy as np
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
def read_in_data():
    data_dir = os.path.join("rundata", "local")
    data_files = os.listdir(data_dir)
    run_data = []
    for data_file in data_files:
        with open(os.path.join(data_dir, data_file), encoding="utf-8") as file:
            metadata = data_file[:-4].split("_")
            if metadata[0][:3] != "exp":
                continue
            experiment_number = int(metadata[0][3:])
            number_runs = int(metadata[2])
            hadamard = len(metadata) == 4

            curr_data = np.zeros(number_runs)
            idx = 0
            for line in file.readlines():
                if "Error:" in line:
                    curr_data[idx] = float(line.split(" ")[2])
                    idx += 1
            
            run_data.append({
                "experiment_number": experiment_number,
                "number_runs": number_runs,
                "hadamard": hadamard,
                "data": curr_data
            })
    return run_data

def read_in_data_file(file_path):
    data_dir = os.path.join("rundata", "local")
    run_data = []
    with open(os.path.join(data_dir, file_path), encoding="utf-8") as file:
        curr_data = []
        experiment_number = 0
        number_runs = None
        hadamard = None

        for line in file:
            if "Index:" in line:
                # Process the previous experiment data if it exists
                if curr_data:
                    run_data.append({
                        "experiment_number": experiment_number + 1,
                        "number_runs": 512,
                        "hadamard": hadamard,
                        "data": np.array(curr_data)
                    })
                    curr_data = []
                continue

            if "Hadamard:" in line:
                hadamard = line.split(":")[1].strip().lower() == 'true'
                continue

            if "Experiment:" in line:
                experiment_number = int(line.split(":")[1].strip())
                continue

            # Process the error data lines
            if "Error:" in line:
                curr_data.append(float(line.split(" ")[2]))

        # Don't forget to add the last experiment's data
        if curr_data:
            run_data.append({
                "experiment_number": experiment_number + 1,
                "number_runs": number_runs,
                "hadamard": hadamard,
                "data": np.array(curr_data)
            })

    return run_data

exp_names = ['000, 001', '000, 110', '000, 111', '000, 010, 100', '000, 011, 100', '000, 011, 110', 'General Unitary']

def plot_experiments(run_data, output_dir):
    # Group data by experiment number

    plt.rcParams['lines.linewidth'] = 20  # Sets the line width
    
    grouped_data = {}
    for run in run_data:
        exp_num = run['experiment_number']
        key = (exp_num, run['hadamard'])
        if key not in grouped_data:
            grouped_data[key] = []
        grouped_data[key].extend(run['data'])

        # Create box and whisker plots
    for exp_num in set(key[0] for key in grouped_data):
        hadamard_data = grouped_data.get((exp_num, True), [])
        no_hadamard_data = grouped_data.get((exp_num, False), [])

        data_to_plot = []
        labels = []

        if hadamard_data:
            data_to_plot.append(hadamard_data)
            labels.append('Randomized')

        if no_hadamard_data:
            data_to_plot.append(no_hadamard_data)
            labels.append('Unrandomized')

        if not data_to_plot:  # Skip if no data is available
            continue

        plt.figure(figsize=(4, 4))
        plt.boxplot(data_to_plot, labels=labels, widths=0.4)
        plt.ylabel('Error Percentage')
        plt.ylim(0, 5) if no_hadamard_data[0] < 10 else plt.ylim(0, 100)
        if (hadamard_data is not None) ^ (no_hadamard_data is not None):
            plt.title(f'{exp_names[exp_num - 1]}')
        else:
            plt.title(f'{exp_names[exp_num - 1]}')
        plt.savefig(os.path.join(output_dir, f'Experiment_{exp_num}_Comparison.png'), bbox_inches='tight')
        plt.close()



In [7]:
# run_data = read_in_data_file("log_2023_11_22T_10_52_58.txt")
run_data = read_in_data()
output_dir = os.path.join("rundata", "charts")
plot_experiments(run_data, output_dir)